In [1]:
import json
import time
import numpy as np
import paho.mqtt.client as mqtt
from fmi_mlc import fmi_gym
from config import parameter as PARAMS
from mqtt_publisher import MQTTPublisher

# Buffer para recibir la última acción
action_buffer = None

def on_connect(client, userdata, flags, rc):
    print(f"✅ Controlled Sim — Connected (rc={rc})")
    client.subscribe("building/action", qos=1)
    print("✅ Controlled Sim — Subscribed to topic: building/action")

def on_action(client, userdata, msg):
    global action_buffer
    payload = json.loads(msg.payload)
    # Aceptamos dict{name: value} o lista
    if isinstance(payload, list):
        action_buffer = {name: payload[i] for i, name in enumerate(PARAMS['action_names'])}
    elif isinstance(payload, dict):
        action_buffer = payload
    else:
        print("⚠️ Controlled Sim — Payload inesperado:", payload)
        return

# 1) Cliente MQTT
mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_action
mqtt_client.connect("mosquitto", 1883)
mqtt_client.loop_start()

# 2) Publisher para Influx (vía MQTT)
pub = MQTTPublisher(host="mosquitto", port=1883)

# 3) Crear y resetear el entorno
env = fmi_gym(PARAMS)
obs = env.reset()
last = env.data.iloc[-1].to_dict()
ts = last['time']

# 4) Publicar observación inicial al controlador y a Influx RL
obs_ctrl = {n: last[n] for n in PARAMS['observation_names']}
mqtt_client.publish("building/observation", json.dumps(obs_ctrl), qos=1)
print(f"→ Published initial observation: {obs_ctrl}")
pub.publish_observations_rl(obs_ctrl, ts)
print(f"→ Published initial RL observation at ts={ts}")

# 5) Bucle principal
done = False
while not done:
    # 5a) Esperar a que action_buffer sea dict
    while action_buffer is None:
        time.sleep(0.001)

    # 5b) Consumir la acción
    act_dict = action_buffer
    action_buffer = None

    # 5c) Publicar acciones con publish_actions_rl
    pub.publish_actions_rl(act_dict, ts)

    # 5d) Avanzar el entorno
    act = np.array([act_dict[n] for n in PARAMS['action_names']], dtype=np.float32)
    obs, reward, done, _ = env.step(act)
    last = env.data.iloc[-1].to_dict()
    ts = last['time']

    # 5e) Publicar siguiente observación al controlador
    obs_ctrl = {n: last[n] for n in PARAMS['observation_names']}
    mqtt_client.publish("building/observation", json.dumps(obs_ctrl), qos=1)

    pub.publish_observations_rl(obs_ctrl, ts)

    # Recompensas individuales (cada PPD)
    reward_dict = {r: last[r] for r in PARAMS['reward_names']}
    pub.publish_rewards_rl(reward_dict, ts)

# 6) Cierre
print("✅ Controlled Sim — Episode finished.")
pub.close()
mqtt_client.loop_stop()
mqtt_client.disconnect()


/tmp/ipykernel_68127/947897052.py:30: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


✅ Controlled Sim — Connected (rc=0)
✅ Controlled Sim — Subscribed to topic: building/action
Reading input and weather file for preprocessor program.
The IDF version of the input file ///tmp/JModelica.org/jm_tmphfnvponn//resources//Res_flat1.idf starts with 9
Successfully finish reading weather file.
This is the Begin Month: 7
This is the Day of the Begin Month: 2
This is the End Month: 7
This is the Day of the End Month: 31
Day of week was left blank in input file.
This is the New Day of Week:  
Running EPMacro...
ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.015
EnergyPlus Starting
EnergyPlus, Version 9.6.0-4b123cf80f, YMD=2025.06.17 16:12
Initializing Response Factors
Calculating CTFs for "PAVIMENTO INTERNO DI PROGETTO"
Calculating CTFs for "_FLATROOF_MOD"
Calculating CTFs for "115MM SINGLE LEAF BRICK (PLASTERED BOTH SIDES)"
Calculating CTFs for "_WALL_MOD"
Calculating CTFs for "PARETE SEMI-ESPOSTA DI PROGETTO"
Calculating CTFs for "PAVIMENTO 

[WARNING][Slave] [warning][FMU status:Warning] fmiInitializeSlave: The StopTimeDefined parameter is set to 0. This is not valid. EnergyPlus FMU requires a stop time and will use the stop time 18316800.000000 which is provided.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The sockfd is 64.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The port number is 35501.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: This hostname is af345983e163.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: TCPServer Server waiting for clients on port: 35501.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of input variables is 3.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of output variables is 7.

[INFO][Slave] [ok][FMU status:OK] Get input file from resource folder ///tmp/JModelica.org/jm_tmphfnvponn//resources//.

[INFO][Slave] [ok][FMU status:OK] Searching for following pattern .idf

[INFO][Slave] [ok][FMU status:OK] Read directory

Updating Shadowing Calculations, Start Date=07/22/2002
Updating Detailed Daylighting Factors, Start Date=07/22
Continuing Simulation at 07/22/2002 for SENZA TITOLO (01-01:31-12)
✅ Controlled Sim — Episode finished.


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>